In [1]:
import pandas as pd



In [2]:
#!pip install mysql-connector-python

In [82]:
import mysql.connector

from sqlalchemy import create_engine

In [84]:
from getpass import getpass

In [90]:
password = getpass.getpass()

 ········


In [92]:
cnx=mysql.connector.connect(user="root",
                            password=password,
                            host="localhost",
                            database="sakila")

In [94]:
cnx.is_connected()

True

In [96]:
cursor = cnx.cursor()

In [98]:



def rentals_month(cnx, month, year):
    query = f"""
    SELECT r.customer_id, r.rental_date
    FROM rental r
    WHERE MONTH(r.rental_date) = {month} AND YEAR(r.rental_date) = {year};
    """
    try:
        df = pd.read_sql(query, cnx)
        return df
    except Exception as e:
        print(f"Error retrieving rental data for {month}/{year}: {e}")
        return pd.DataFrame()  # Return an empty DataFrame on error

def rental_count_month(df, month, year):
    column_name = f'rentals_{month:02d}_{year}'
    rental_counts = df.groupby('customer_id').size().reset_index(name=column_name)
    return rental_counts

def compare_rentals(df1, df2):
    combined = pd.merge(df1, df2, on='customer_id', how='outer')
    combined['difference'] = combined[df1.columns[1]].fillna(0) - combined[df2.columns[1]].fillna(0)
    return combined


rental_counts_may = pd.DataFrame()
rental_counts_june = pd.DataFrame()


df_may = rentals_month(cnx, 5, 2005)
print("May 2005 Rentals:")
print(df_may.head())  # Display the first few rows


if not df_may.empty:
    rental_counts_may = rental_count_month(df_may, 5, 2005)
    print("\nRental Counts for May 2005:")
    print(rental_counts_may)
else:
    print("No data retrieved for May 2005.")


df_june = rentals_month(cnx, 6, 2005)
print("\nJune 2005 Rentals:")
print(df_june.head())  


if not df_june.empty:
    rental_counts_june = rental_count_month(df_june, 6, 2005)
    print("\nRental Counts for June 2005:")
    print(rental_counts_june)
else:
    print("No data retrieved for June 2005.")


if not rental_counts_may.empty and not rental_counts_june.empty:
    rental_comparison = compare_rentals(rental_counts_may, rental_counts_june)
    # Step 7: Display the comparison DataFrame
    print("\nRental Comparison between May and June 2024:")
    print(rental_comparison)
else:
    print("No data available for comparison.")


if cnx.is_connected():
    cnx.close()
    print("Database connection closed.")


May 2005 Rentals:
   customer_id         rental_date
0          130 2005-05-24 22:53:30
1          459 2005-05-24 22:54:33
2          408 2005-05-24 23:03:39
3          333 2005-05-24 23:04:41
4          222 2005-05-24 23:05:21

Rental Counts for May 2005:
     customer_id  rentals_05_2005
0              1                2
1              2                1
2              3                2
3              5                3
4              6                3
..           ...              ...
515          594                4
516          595                1
517          596                6
518          597                2
519          599                1

[520 rows x 2 columns]

June 2005 Rentals:
   customer_id         rental_date
0          416 2005-06-14 22:53:33
1          516 2005-06-14 22:55:13
2          239 2005-06-14 23:00:34
3          285 2005-06-14 23:07:08
4          310 2005-06-14 23:09:38

Rental Counts for June 2005:
     customer_id  rentals_06_2005
0              1 

C:\Users\ngaut\AppData\Local\Temp\ipykernel_3580\1182306190.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, cnx)
C:\Users\ngaut\AppData\Local\Temp\ipykernel_3580\1182306190.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, cnx)
